# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [6]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [7]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [9]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [10]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [155]:
#import pandas for dataframe
import pandas as pd

#create dataframe and declare index counter
mort_df = pd.DataFrame(columns=('country','code','infant_mortality'))
i = 0

#parse document and add rows to dataframe
document = ET.parse( './data/mondial_database.xml' )
for element in document.iterfind('country'):
    #check for nulls and do not add those to the df, cast columns as string and float
    if element.find('infant_mortality') is not None:
        mort_df.loc[i]=([str(element.find('name').text),str(element.get('car_code')),float(element.find('infant_mortality').text)])
        i=i+1


In [255]:
#the filled dataframe with no nulls
mort_df.drop_duplicates()
print (mort_df)

                    country  code  infant_mortality
0                   Albania    AL             13.19
1                    Greece    GR              4.78
2                 Macedonia    MK              7.90
3                    Serbia   SRB              6.16
4                   Andorra   AND              3.69
5                    France     F              3.31
6                     Spain     E              3.33
7                   Austria     A              4.16
8            Czech Republic    CZ              2.63
9                   Germany     D              3.46
10                  Hungary     H              5.09
11                    Italy     I              3.31
12            Liechtenstein    FL              4.33
13                 Slovakia    SK              5.35
14                 Slovenia   SLO              4.04
15              Switzerland    CH              3.73
16                  Belarus    BY              3.64
17                   Latvia    LV              7.91
18          

In [264]:
#find the top 10 countries with the smallest infant mortality rate
mort_df.sort_values(by='infant_mortality').head(10)

,country,code,infant_mortality
36,Monaco,MC,1.81
90,Japan,J,2.13
109,Bermuda,BERM,2.48
34,Norway,N,2.48
98,Singapore,SGP,2.53
35,Sweden,S,2.60
8,Czech Republic,CZ,2.63
72,Hong Kong,HONX,2.73
73,Macao,MACX,3.13
39,Iceland,IS,3.15


In [276]:
#create dataframe and declare index counter
cities = pd.DataFrame(columns=('city','population'))
i = 0

for element in document.findall('country'):
    for child in element.iter('city'):
        p = 0
        y = 0
        for population in child.findall('population'):
            #set p from population where year > previous year
            if population.text is not None and population.get('year') is not None and int(population.get('year'))>=y:
                p = int(population.text)
                y = int(population.get('year'))
        #We may hve a city with 0 population, so filter those out
        if child.find('name') is not None and p>0:
            cities.loc[i]=([str(child.find('name').text),p])
            i=i+1


In [277]:
print (cities)

              city  population
0           Tirana    418495.0
1          Shkodër     77075.0
2           Durrës    113249.0
3            Vlorë     79513.0
4          Elbasan     78703.0
5            Korçë     51152.0
6           Kavala     58790.0
7           Athina    664046.0
8         Peiraias    163688.0
9        Peristeri    139981.0
10        Acharnes    106943.0
11           Patra    213984.0
12         Kerkyra    102071.0
13        Ioannina    112486.0
14    Thessaloniki    325182.0
15         Iraklio    173993.0
16          Chania    108642.0
17          Rhodes    115490.0
18           Lamia     75315.0
19        Chalkida    102223.0
20         Larissa    162591.0
21           Volos    144449.0
22          Karyes       233.0
23          Skopje    514967.0
24        Kumanovo    107745.0
25         Beograd   1639121.0
26        Novi Sad    335701.0
27             Niš    257867.0
28       Podgorica    150977.0
29       Prishtine    198214.0
...            ...         ...
3018    

In [278]:
#10 cities with largest population
#cities.nlargest(10,'population')
cities.sort_values(by='population',ascending=False).head(10)

,city,population
1250,Shanghai,22315474.0
706,Istanbul,13710512.0
1420,Mumbai,12442373.0
442,Moskva,11979529.0
1249,Beijing,11716620.0
2592,São Paulo,11152344.0
1251,Tianjin,11090314.0
973,Guangzhou,11071424.0
1466,Delhi,11034555.0
976,Shenzhen,10358381.0


In [279]:
#create dataframe and declare index counter
egroups = pd.DataFrame(columns=('group','population'))
i = 0

for element in document.findall('country'):
    p = 0
    y = 0
    #Get the population of the country just like the cities above
    for population in element.findall('population'):
        if population.text is not None and population.get('year') is not None and int(population.get('year'))>y:
            p = int(population.text)
            y = int(population.get('year'))
    for child in element.findall('ethnicgroup'):
        #check for nulls and do not add those to the df, cast columns desired dtypes.
        #I convert population of group to int because you do not have .2 of a person in reality
        if child.get('percentage') is not None and p>0:
            egroups.loc[i]=([str(child.text),int(p * ((float(child.get('percentage')))/100))])
            i=i+1


In [280]:
print (egroups)

                    group  population
0                Albanian   2660131.0
1                   Greek     84004.0
2                   Greek  10059145.0
3              Macedonian   1322387.0
4                Albanian    519068.0
5                 Turkish     80331.0
6                   Gypsy     55614.0
7                    Serb     37076.0
8                    Serb   5903032.0
9             Montenegrin     64085.0
10              Hungarian    277705.0
11                   Roma     99689.0
12                Bosniak    128171.0
13                  Croat     78327.0
14            Montenegrin    266612.0
15                   Serb    198409.0
16                Bosniak     49602.0
17               Albanian     31001.0
18               Albanian   1595162.0
19                Serbian     86693.0
20                Spanish     33589.0
21               Andorran     25777.0
22             Portuguese      8592.0
23                 French      1562.0
24                African      3905.0
25   Mediter

In [281]:
#sum the population of each group
esum = egroups.groupby('group').sum()

In [282]:
#10 largest ethnic groups
#esum.nlargest(10,'population')
esum.sort_values(by='population',ascending=False).head(10)

,population
group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577343e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [223]:
#create dataframe and declare length variable
#we are going to loop through the xml doc and replace our rows in the dataframe when
#we find a new lenght > previous length
#we should end up with a dataframe that has the country(s) with the one longest river
rivers = pd.DataFrame(columns=('name','country','length'))
l = 0

for river in document.findall('river'):
    #create an empty list to split country codes into
    countries = []
    if river.attrib is not None:
        countries = river.attrib['country'].split()
    if river.find('length') is not None and float(river.find('length').text)>l:
        #We have a river, with a longer length, so let's loop through the country codes and add to df
        l = float(river.find('length').text)
        for code in countries:
            #Let's get the country name from our mort_df dataframe, but first make sure we have that code
            if len(mort_df[mort_df['code'].astype(str).str.contains(code)])>0:
                cname = mort_df.loc[mort_df['code'] == code, 'country'].iloc[0]
                #clear the dataframe
                rivers.iloc[0:0]
                i = 0
                #We now have the river name, length and country so add it to the empty dataframe
                if len(cname)>0:
                    rivers.loc[i]=([str(river.find('name').text),cname,l])
                    i=i+1


In [224]:
print(rivers)

       name country  length
0  Amazonas    Peru  6448.0


In [225]:
#Let's repeat this for lakes and airports
lakes = pd.DataFrame(columns=('name','country','area'))
l = 0

for lake in document.findall('lake'):
    countries = []
    if lake.find('area') is not None and float(lake.find('area').text)>l:
        countries = lake.attrib['country'].split()
        l = float(lake.find('area').text)
        for code in countries:
            if len(mort_df[mort_df['code'].astype(str).str.contains(code)])>0:
                cname = mort_df.loc[mort_df['code'] == code, 'country'].iloc[0]
                lakes.iloc[0:0]
                i = 0
                if len(cname)>0:
                    lakes.loc[i]=([str(lake.find('name').text),cname,l])
                    i=i+1


In [254]:
print (lakes)

          name       country      area
0  Caspian Sea  Turkmenistan  386400.0


In [228]:
airports = pd.DataFrame(columns=('name','country','elevation'))
l = 0

for airport in document.findall('airport'):
    countries = []
    if airport.find('elevation').text is not None and float(airport.find('elevation').text)>l:
        countries = airport.attrib['country'].split()
        l = float(airport.find('elevation').text)
        for code in countries:
            if len(mort_df[mort_df['code'].astype(str).str.contains(code)])>0:
                cname = mort_df.loc[mort_df['code'] == code, 'country'].iloc[0]
                airports.iloc[0:0]
                i = 0
                if len(cname)>0:
                    airports.loc[i]=([str(airport.find('name').text),cname,l])
                    i=i+1


In [229]:
print(airports)

           name  country  elevation
0  El Alto Intl  Bolivia     4063.0


In [240]:
print('The longest river is '+rivers.loc[0]['name']+', in '+rivers.loc[0]['country']+', with a length of '+
      str(rivers.loc[0]['length'])+'\n'+
     'The largest lake is '+lakes.loc[0]['name']+', in '+lakes.loc[0]['country']+', with an area of '+
      str(lakes.loc[0]['area'])+'\n'+
     'The highest airport is '+airports.loc[0]['name']+', in '+airports.loc[0]['country']+', with an elevation of '+
      str(airports.loc[0]['elevation']))

The longest river is Amazonas, in Peru, with a length of 6448.0
The largest lake is Caspian Sea, in Turkmenistan, with an area of 386400.0
The highest airport is El Alto Intl, in Bolivia, with an elevation of 4063.0
